First, lets install Ivy via pip and import it alongside the other frameworks we'll be using.

In [ ]:
!pip install -q ivy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00


In [ ]:
import ivy
import torch
import tensorflow as tf
import numpy as np

Here we create a basic TensorFlow Keras model containing a single LSTM layer as an example.

We can then convert this model to PyTorch by transpiling with ivy.transpile and providing some input arguments for the model.

In [ ]:
with tf.device("/CPU:0"):
  sample_input = tf.random.uniform((5, 2, 40))

  # build the lstm keras model
  tf_lstm = tf.keras.Sequential([tf.keras.layers.LSTM(40)])
  tf_lstm.build(sample_input.shape)

  # transpile to torch
  torch_lstm = ivy.transpile(tf_lstm, source="tensorflow", to="torch", args=(sample_input,))

/usr/local/lib/python3.10/dist-packages/ivy/utils/exceptions.py:383: UserWarning: The current backend: 'tensorflow' does not support inplace updates natively. Ivy would quietly create new arrays when using inplace updates with this backend, leading to memory overhead (same applies for views). If you want to control your memory management, consider doing ivy.set_inplace_mode('strict') which should raise an error whenever an inplace update is attempted with this backend.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:164: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return _transpile(
/usr/local/lib/python3.10/dist-packages/ivy/compiler/compiler.py:164: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  return _transpile(


Now we've transpiling the model to PyTorch, lets verify that the results produced by the new PyTorch model are identical to those produced by the original Keras model.

We'll use an input tensor with different shape to the input the model was transpiled with, to verify that the transpiled model is compatible with dynamic input shapes.

In [ ]:
# identical input tensors for torch and tf
torch_input = torch.rand((10, 100, 40))
tf_input = tf.constant(torch_input.cpu().detach().numpy())

# compile the original tensorflow model
tf_lstm = tf.function(tf_lstm)

# get output of the original and transpiled models
tf_output = tf_lstm(tf_input)
torch_output = torch_lstm(torch_input)

# verify the outputs are the same (with some tolerance)
np.allclose(tf_output[0].numpy(), torch_output[0].cpu().detach().numpy(), atol=1e-6)

True

Finally, lets benchmark the transpiled torch model compared to the original. Here we benchmark on both CPU and GPU over 1000 inference runs.

In [ ]:
# run some benchmarks

from time import perf_counter

N_RUNS = 1000

tf_inputs = tf.random.normal([10, 100, 40])
torch_inputs = torch.from_numpy(tf_inputs.numpy())


# benchmark on CPU
with tf.device("/CPU:0"):
  torch_lstm = torch_lstm.to("cpu")

  tf_inputs = tf.random.normal([10, 100, 40])
  torch_inputs = torch.from_numpy(tf_inputs.numpy()).to("cpu")

  # time the tf lstm
  s = perf_counter()
  for _ in range(N_RUNS):
    tf_lstm(tf_inputs)
  tf_time = round(perf_counter() - s, 4)

  # time the transpiled torch lstm
  s = perf_counter()
  for _ in range(N_RUNS):
    torch_lstm(torch_inputs)
  torch_time = round(perf_counter() - s, 4)

  print(f'(CPU)  tensorflow lstm time: {tf_time} seconds  transpiled torch lstm time: {torch_time} seconds')

  cpu_speedup = round(tf_time / torch_time, 3)


print(f'\ntranspiled torch lstm is {cpu_speedup}x faster than tensorflow\'s lstm on CPU')

(CPU)  tensorflow lstm time: 5.5017 seconds  transpiled torch lstm time: 2.1101 seconds
(GPU)  tensorflow lstm time: 1.7519 seconds  transpiled torch lstm time: 0.901 seconds

transpiled torch lstm is 2.607x faster than tensorflow's lstm on CPU
transpiled torch lstm is 1.944x faster than tensorflow's lstm on GPU


We can see that the results of the transpiled PyTorch model are significantly faster than the original Keras model on both CPU and GPU :)